In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from fastai import *
from fastai.io import *
from fastai.conv_learner import *
from fastai.column_data import *
torch.cuda.set_device(0)
import numpy as np
import pandas as pd
from pathlib import Path

### Load Data

In [4]:
PATH = Path('/mnt/data/extracts/rnns/')
txt = (PATH/'nietzsche.txt').read_text()

### Create Vocab

In [6]:
chars = set(txt)
padding_char = "\0"
chars.add(padding_char)
chars = sorted(list(chars))
c2i = {c:i for i, c in enumerate(chars)}
i2c = {i:c for i, c in enumerate(chars)}
vocab_size = len(c2i)

### Convert Text

In [7]:
inp = [c2i[c] for c in txt]

In [8]:
x1 = [inp[i] for i in range(0, len(inp) - 3, 3)]
x2 = [inp[i] for i in range(1, len(inp) - 3, 3)]
x3 = [inp[i] for i in range(2, len(inp) - 2, 3)]
x4 = [inp[i] for i in range(3, len(inp) - 1, 3)]

In [9]:
x = np.concatenate([np.vstack(x1), np.vstack(x2), np.vstack(x3)], axis=1)
y = np.stack(x4)

### Build 3 char Model

In [82]:
class char3_rnn(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super(char_rnn, self).__init__()
        self.emb = nn.Embedding(vocab_size, n_fac)
        self.input = nn.Linear(n_fac, n_fac)
        self.hidden = nn.Linear(n_fac, n_fac)
        self.output = nn.Linear(n_fac, vocab_size)
        
    def forward(self, c1, c2, c3):
        x1 = F.relu(self.input(self.emb(c1)))
        x2 = F.relu(self.input(self.emb(c2)))
        x3 = F.relu(self.input(self.emb(c3)))
        
        h = V(torch.zeros(x1.size()))
        h = F.tanh(self.hidden(h + x1))
        h = F.tanh(self.hidden(h + x2))
        h = F.tanh(self.hidden(h + x3))
        return F.log_softmax(self.output(h))

In [83]:
md = ColumnarModelData.from_arrays(PATH, [-1], xs=x, y=y, bs=14)

In [84]:
it = iter(md.trn_dl)
*c, c4 = next(it)

In [85]:
ch = char3_rnn(vocab_size, 256, 256).cuda()
opt = optim.Adam(ch.parameters(), lr=0.001)
crit = F.nll_loss

In [86]:
fit(ch, md, 1, opt, crit)

 10%|█         | 1471/14307 [00:07<01:08, 188.20it/s, loss=2.34]
epoch      trn_loss   val_loss                                   
    0      1.911082   0.698806  



[0.6988058090209961]

In [352]:
def predict(mod, word):
    x = [c2i[c] for c in word]
    probs = mod(*V(x)).data.cpu()
    idx = np.argmax(probs)
    return i2c[idx]

In [139]:
predict(ch, 'cit')

'y'

### Build RNN

In [273]:
cs = 7
x = [[inp[i + j] for j in range(0, cs)] for i in range(0, len(inp) - cs - 7, cs)]
y = [inp[i] for i in range(cs, len(inp) - cs, cs)]
x = np.array(x)
y = np.stack(y)

In [360]:
class char_rnn(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super(char_rnn, self).__init__()
        self.n_hidden = n_hidden
        self.emb = nn.Embedding(vocab_size, n_fac, padding_idx=0)
        self.inp = nn.Linear(n_fac, n_hidden)
        self.hidden = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, self.n_hidden).cuda())
        for c in cs:
            x = F.relu(self.inp(self.emb(c)))
            h = F.tanh(self.hidden(h + x))
        return F.log_softmax(self.out(h))

In [361]:
cr = char_rnn(vocab_size, 50, 256).cuda()

In [16]:
md = ColumnarModelData.from_arrays(PATH, [-1], x, y, bs=512)

In [18]:
opt = optim.Adam(cr.parameters(), lr=0.001)
crit = F.nll_loss

NameError: name 'cr' is not defined

In [364]:
fit(cr, md, 1, opt, crit)

epoch      trn_loss   val_loss                              
    0      2.413876   0.779971  



[0.779970645904541]

In [365]:
fit(cr, md, 1, opt, crit)

epoch      trn_loss   val_loss                              
    0      2.131076   0.492571  



[0.4925713539123535]

In [366]:
fit(cr, md, 1, opt, crit)

epoch      trn_loss   val_loss                              
    0      1.981934   0.371334  



[0.3713340759277344]

In [346]:
opt = optim.Adam(cr.parameters(), lr=0.01)
fit(cr, md, 1, opt, crit)

epoch      trn_loss   val_loss                              
    0      3.983818   3.950012  



[3.95001220703125]

In [383]:
predict(cr, 'for thos')

'e'

### Build RNN using PyTorch

In [13]:
class pytorch_rnn(nn.Module):
    def __init__(self, vocab_size, n_fac, n_inp, n_hidden):
        super(pytorch_rnn, self).__init__()
        self.n_hidden = n_hidden
        self.n_inp = n_inp
        self.emb = nn.Embedding(vocab_size, n_fac)
        self.inp = nn.Linear(n_fac, n_inp)
        self.rnn = nn.RNN(n_hidden + n_inp, n_hidden, 1)
    
    def forward(self, *cs):
        pdb.set_trace()
        x = F.relu(self.inp(self.emb(cs)))

In [23]:
ptr = pytorch_rnn(vocab_size, 50, 128, 256)
md = ColumnarModelData.from_arrays(PATH, [-1], x, y, bs=10)
opt = optim.Adam(ptr.parameters(), lr=0.001)
crit = F.nll_loss

In [25]:
fit(ptr, md, 1, opt, crit)

  0%|          | 0/20030 [00:00<?, ?it/s]> <ipython-input-13-f8ab59345393>(12)forward()
-> x = F.relu(self.inp(self.emb(cs)))

(Pdb) l
  7  	        self.inp = nn.Linear(n_fac, n_inp)
  8  	        self.rnn = nn.RNN(n_hidden + n_inp, n_hidden, 1)
  9  	
 10  	    def forward(self, *cs):
 11  	        pdb.set_trace()
 12  ->	        x = F.relu(self.inp(self.emb(cs)))
[EOF]
(Pdb) cs
(Variable containing:
 68
  9
 60
 65
 38
  2
 58
  2
 73
 69
[torch.cuda.LongTensor of size 10 (GPU 0)]
, Variable containing:
 65
  9
 74
  2
 44
 62
 71
 73
 68
 73
[torch.cuda.LongTensor of size 10 (GPU 0)]
, Variable containing:
 72
 54
 62
 72
  1
 72
  2
 61
 10
  2
[torch.cuda.LongTensor of size 10 (GPU 0)]
)
(Pdb) self.emb(cs)
*** AttributeError: 'tuple' object has no attribute 'dim'
(Pdb) type(cs)
<class 'tuple'>
(Pdb) q


BdbQuit: 